# 일반 구매 차감 
1000만 세톱 데이터에 대해 일반 구매 차감 연산 실행한 결과  
 - 전체 : 94 초  
 - 세션 생성 : 23 초  
 - 차감 계산 : 37 초 (14초.캐시)    
 - 스냅샷 저장 : 32 초  
 - 결과 확인 : 2 초  
 - 요약 결과 생성 : 
 - 요약 rdb 기록 : 


In [36]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [37]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = '03-01_plain-ad-calc'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet'

In [38]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '2g')\
    .config('spark.num.executors', '2')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '4g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [6]:
!ls -l /hive-bin/lib | grep mysql

-rw-r--r-- 1 root root   1006906 Mar 17 01:30 mysql-connector-java-5.1.49-bin.jar
-rw-r--r-- 1 root staff    10476 Nov 15  2018 mysql-metadata-storage-0.12.0.jar


In [39]:
%%time
spark = spark_creation()

CPU times: user 17.8 ms, sys: 9.73 ms, total: 27.5 ms
Wall time: 16 s


/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## 전체 구매 차감  
모든 세탑에 (지정 비율 * 구매 단위) 값을 차감  
- 인벤토리 기준 정보 조회  
- setop 별 비율대로 구매 시간을 반영해서(1 미만 값 등 추가 고민 필요) 차감 데이터 생성 
- 인벤시간에서 차감 데이터를 차감  
- 차감 결과를 이용해 시간, 지역, 타겟별 잔여 리포트 생성  
  
원본 비율을 계속 적용하면 남는 시간이 부족해서 차감 안되는 일이나, 처음부터 비율이 낮아서 판매시간 곱해도 1이 안 되는 오차가 계속 발생할 것으로 추정  
- (구매시간 * )잔여시간 계산 값에 일정비율?을 적용해야 차감총액이 구매시간이 될지를 찾아서, 일정비율을 적용하는 방법   
- 비율*구매시간 결과 중 정수 계산으로 1 이상의 총합이 구매시간이 되도록 비율 가중치를 찾는 방법  
  
***임시처리***  
- 반올림으로 차감 수량 결정(오차 일단 무시) 
- 부족분 분배...  (비율, 남은 수량 으로 빠르게 계산할 방법...)  

## 중요  
- 오차 범위가 큰 부족분 먼저 처리  
- 비율 오차 처리는 내림 계산을 기본으로 나머지 차액을 회귀식으로 최소오차 비율 찾아서 일괄 적용.   
 -> 잔여가 1 남지 않은 수량을 제외하고 계산...  
  - 1. 내림 결과,  2. 부족분 계산, 3. 비율 오차 처리  
  - 1. 총량 계산 
  - 2. 부족 여부 판단 (6개월분 각각? 전체? 판단 필요)   
  - 3. (반올림) 비율 계산 결과 적용 - (-) 가능  
  - 4. 총량 0이 아닌 세톱 모두 조회  
  - 5. 0<x 인 로우만 조회해서 남은 비율 계산.  
  - 6. (-) 수량 * 전체 잔여 * 5 비율 적용. -> (-)는 모두 0이 되고, 나머지는 추가 차감.  
  - 6개월분 모두 적용  

  
>- 내림값 총합에서 남는 값을 
>- 차감 부족분을 어떻게 분배할 지 ...  

In [40]:
%%time
## 인벤 기준 정보 조회  
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
spark.catalog.cacheTable("setop_view")
spark.catalog.isCached('setop_view')

CPU times: user 4.14 ms, sys: 0 ns, total: 4.14 ms
Wall time: 2.41 s


True

In [50]:
%%time
id = "ST_A_000000%"
qty = 2
df_tmp = spark.sql(str.format("select setop, inv_val_01 *({}/100.0) qty from setop_view where setop like '{}'", qty, id))
#df_tmp.to_table('inven.my_table', partition_cols='setop')
df_tmp.createOrReplaceTempView('setop_temp')
df_tmp.summary().show()

+-------+------------+-------------+
|summary|       setop|          qty|
+-------+------------+-------------+
|  count|          10|           10|
|   mean|        null|20.0000000000|
| stddev|        null|          0.0|
|    min|ST_A_0000000|    20.000000|
|    25%|        null|         20.0|
|    50%|        null|         20.0|
|    75%|        null|         20.0|
|    max|ST_A_0000009|    20.000000|
+-------+------------+-------------+

CPU times: user 1.53 ms, sys: 3.15 ms, total: 4.69 ms
Wall time: 1.44 s


In [65]:
%%time
# inv_val_01*({}/100.0)
all_sql = """
select setop, sum(inv_val_01) inv_val_01 
from 
(
    select setop, inv_val_01 from setop_view where setop like 'ST_A_000000%'
    UNION ALL 
    select setop, -qty inv_val_01 from setop_temp 
) 
group by setop 
"""
# spark.sql(all_sql).show()
# spark.sql(all_sql).summary().show()
spark.sql(all_sql).summary().collect()

CPU times: user 9.28 ms, sys: 1.09 ms, total: 10.4 ms
Wall time: 4.37 s


[Row(summary='count', setop='10', inv_val_01='10'),
 Row(summary='mean', setop=None, inv_val_01='980.0000000000'),
 Row(summary='stddev', setop=None, inv_val_01='0.0'),
 Row(summary='min', setop='ST_A_0000000', inv_val_01='980.000000'),
 Row(summary='25%', setop=None, inv_val_01='980.0'),
 Row(summary='50%', setop=None, inv_val_01='980.0'),
 Row(summary='75%', setop=None, inv_val_01='980.0'),
 Row(summary='max', setop='ST_A_0000009', inv_val_01='980.000000')]

In [60]:
%%time
# inv_val_01*({}/100.0)
all_sql = """
select setop_view.setop setop, setop_view.inv_val_01+b.inv_val_01 inv_val_01 
--select  setop_view.setop setop, setop_view.inv_val_01, b.inv_val_01 inv_val_01 
from setop_view LEFT OUTER JOIN ( select setop, -qty inv_val_01 from setop_temp) b 
ON (setop_view.setop = b.setop) 
where setop_view.setop like 'ST_A_00000%'
"""
spark.sql(all_sql).summary().show()
# spark.sql(all_sql).show(100)

+-------+------------+--------------+
|summary|       setop|    inv_val_01|
+-------+------------+--------------+
|  count|         100|            10|
|   mean|        null|980.0000000000|
| stddev|        null|           0.0|
|    min|ST_A_0000000|    980.000000|
|    25%|        null|         980.0|
|    50%|        null|         980.0|
|    75%|        null|         980.0|
|    max|ST_A_0000099|    980.000000|
+-------+------------+--------------+

CPU times: user 803 µs, sys: 4.69 ms, total: 5.49 ms
Wall time: 4.65 s


In [6]:
# row = spark.sql("select * from setop_view limit 5").select("setop").take(1)
# row[0].asDict()["setop"]

[Row(setop='ST_B_0499136')]

In [9]:
%%time
# 1) 총량 계산 : 최초 24 초. 재실행 시 1초 미만. 
sql_total = "select sum(inv_val_01), sum(inv_val_02), sum(inv_val_03), sum(inv_val_04), sum(inv_val_05), sum(inv_val_06) from setop_view"
spark.sql(sql_total).show()

+---------------+---------------+---------------+---------------+---------------+---------------+
|sum(inv_val_01)|sum(inv_val_02)|sum(inv_val_03)|sum(inv_val_04)|sum(inv_val_05)|sum(inv_val_06)|
+---------------+---------------+---------------+---------------+---------------+---------------+
|    10000000000|    10000000000|    10000000000|    10000000000|    10000000000|    10000000000|
+---------------+---------------+---------------+---------------+---------------+---------------+

CPU times: user 10.2 ms, sys: 982 µs, total: 11.2 ms
Wall time: 26.2 s


In [35]:
#!pip install numpy pandas
#!python3 -m pip install -U pip
# !pip install matplotlib

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)
     |################################| 3.1 MB 1.8 MB/s            


In [50]:
# # 2) 부족 여부 판단. 생략 ... 
# # 아래는 총합 100% 근사한 분포수 생성  
# import numpy as np
# import matplotlib.pyplot as plt  

# cnt = 10000*1000
# rates = np.random.normal(loc=2/cnt*50, scale=1/cnt, size=cnt)
# plt.hist(rates)
# print(sum(rates))

In [10]:
%%time 
# 3) 비율 계산 반올림  
pcnt = 1000*1000*10 # 100 이어야 하지만 1천만건 하드 코딩 비율 조정 위해 임시값. 
sql_total = f"select setop, -(inv_val_01*inv_rate_01/{pcnt}) inv_val_01, -(inv_val_02*inv_rate_02/{pcnt}) inv_val_02, -(inv_val_03*inv_rate_03/{pcnt}) inv_val_03\
, -(inv_val_04*inv_rate_04/{pcnt}) inv_val_04, -(inv_val_05*inv_rate_05/{pcnt}) inv_val_05, -(inv_val_06*inv_rate_06/{pcnt}) inv_val_06 from setop_view"
sql_all = f"select sum(inv_val_01) inv_val_01, sum(inv_val_02) inv_val_02, sum(inv_val_03) inv_val_03\
, sum(inv_val_04) inv_val_04, sum(inv_val_05) inv_val_05, sum(inv_val_06) inv_val_06 from ({sql_total}) as AL"

spark.sql(sql_all).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          inv_val_01|          inv_val_02|          inv_val_03|          inv_val_04|          inv_val_05|          inv_val_06|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|-0.00999999977654...|-0.00999999977654...|-0.00999999977654...|-0.00999999977654...|-0.00999999977654...|-0.00999999977654...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

CPU times: user 0 ns, sys: 4.17 ms, total: 4.17 ms
Wall time: 2.48 s


In [11]:
%%time
# 3) 차감 계산  
sql_calc = f"select setop, sum(inv_val_01) inv_val_01 , sum(inv_val_02) inv_val_02, sum(inv_val_03) inv_val_03\
, sum(inv_val_04) inv_val_04 , sum(inv_val_05) inv_val_05, sum(inv_val_06) inv_val_06 \
from ( \
{sql_total} \
UNION ALL \
select setop, inv_val_01, inv_val_02, inv_val_03, inv_val_04, inv_val_05, inv_val_06 from setop_view \
) as AL \
group by setop"
#spark.sql(sql_calc).count()
spark.sql(sql_calc).createOrReplaceTempView("tbl_subtracted")
# 차감 스냅샷 기록  
# ...  

CPU times: user 2.43 ms, sys: 450 µs, total: 2.88 ms
Wall time: 159 ms


In [12]:
%%time
# 4) 총량 0이 아닌 세톱 조회  
spark.sql("select * from tbl_subtracted where inv_val_01>0 or inv_val_02>0 or \
inv_val_03>0 or inv_val_04>0 or inv_val_05>0 or inv_val_06>0 ").createOrReplaceTempView("tbl_remains")

CPU times: user 188 µs, sys: 2.92 ms, total: 3.11 ms
Wall time: 76.5 ms


In [13]:
%%time
# 5) 남은 비율 계산 : 42 초    
# 5-1) (-) 수량. 일단 하드 코딩 
#spark.sql("select 1000 inv_val_01, 900 inv_val_02, 950 inv_val_03, 850 inv_val_04, 810 inv_val_05, 890 inv_val_06").createOrReplaceTempView("remains_qty")
remains = [1000, 900, 950, 850, 880, 890]
totals = """select 
sum(inv_val_01) inv_val_01, sum(inv_val_02) inv_val_02, sum(inv_val_03) inv_val_03
, sum(inv_val_04) inv_val_04, sum(inv_val_05) inv_val_05, sum(inv_val_06) inv_val_06 
from tbl_remains
"""
spark.sql(totals).createOrReplaceTempView("totals")
# 5-2) 도수 / 남은 총량. 남은 수량을 차감할 비율. 
sql_minus_remains = f"""
select setop
, -{remains[0]}*tbl_remains.inv_val_01/totals.inv_val_01 inv_val_01, -{remains[1]}*tbl_remains.inv_val_02/totals.inv_val_02 inv_val_02 
, -{remains[2]}*tbl_remains.inv_val_03/totals.inv_val_03 inv_val_03, -{remains[3]}*tbl_remains.inv_val_04/totals.inv_val_04 inv_val_04 
, -{remains[4]}*tbl_remains.inv_val_05/totals.inv_val_05 inv_val_05, -{remains[5]}*tbl_remains.inv_val_06/totals.inv_val_06 inv_val_06 
from tbl_remains cross join totals 
"""
spark.sql(sql_minus_remains).createOrReplaceTempView("minus_remains")

#spark.sql("select * from minus_remains limit 5").show()

CPU times: user 2.5 ms, sys: 477 µs, total: 2.97 ms
Wall time: 113 ms


In [14]:
%%time
# 6) 추가 차감 및 결과 스냅샷 기록 : 70 초   
sql_result = """
select setop
, sum(inv_val_01) inv_val_01 , sum(inv_val_02) inv_val_02
, sum(inv_val_03) inv_val_03 , sum(inv_val_04) inv_val_04
, sum(inv_val_05) inv_val_05 , sum(inv_val_06) inv_val_06
from 
(
select * 
from tbl_subtracted 
UNION ALL 
select * 
from minus_remains
) as A 
group by setop 
"""

result = spark.sql(sql_result)
result.write.save(path="result", format=file_format, mode='overwrite')

CPU times: user 23.7 ms, sys: 21.5 ms, total: 45.2 ms
Wall time: 2min 26s


In [17]:
%%time
cnt = spark.read.format(file_format).load('result').count()
result.createOrReplaceTempView("result")
print(f'Row count = {cnt:,}')

Row count = 10,000,000
CPU times: user 4.12 ms, sys: 930 µs, total: 5.05 ms
Wall time: 2.39 s


In [ ]:
# 스냅샷에서 차감 리포트 생성  
# ...  

In [19]:
%%time
# rdb 쓰기 테스트 
# conf = SparkConf()  # create the configuration
#conf.set("spark.jars", "/path/to/postgresql-connector-java-someversion-bin.jar")  # set the spark.jars
# --driver-class-path postgresql-9.4.1207.jar --jars postgresql-9.4.1207.jar
# /hadoop/share/hadoop/yarn/lib 에 mysql jdbc.jar 넣는다.  
props = {"driver":"com.mysql.jdbc.Driver"}
db_url = "jdbc:mysql://rdb/test_jdbc?user=jdbc&password=jdbc"
tbl = "from_spark"
result.cache()
rdb = spark.sql('select * from result limit 10')
rdb.write.jdbc(db_url, tbl, mode='append', properties=props)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.6/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# jdbcDF = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql:dbserver") \
#     .option("dbtable", "jdbc:postgresql:dbserver") \
#     .load()

In [35]:
spark.stop()